#### Read Me
###### This page could be used to generate parking sequences. 
###### The last function, 'easyRun()', is the only one that users have to use to run an example. 
###### easyRun() takes in 2 parameters: the length vector and z. It prints out all possible parking configurations, preference vectors, and prime parking sequences.  
###### You should load all the chunks and use this command to run examples: 
###### length_vector = [1,1,3]
###### z = 1
###### easyRun(length_vector, z)
###### If you want to check which parking configuration corresponds to which preference vectors, you could add True as the third parameter to the function, just like this:
###### easyRun(length_vector, z, True)

In [14]:
# import libraries
from itertools import permutations
from itertools import product

import numpy as np

In [15]:
# give the number of Parking Sequences of a given length vector
def numPS(input_y, z, n):
    '''
    takes in three parameters: y is the length vector, z is the length of the trailer plus 1, and n is the number of cars
    returns the number of parking sequences
    >>> numPS([1,1,2],1,3)
    16
    '''
    # base case
    if len(input_y)==1:
        return z
    # recursive step
    else:
        # remove the last entry
        y = input_y.copy()
        y.remove(y[-1])
        # sum up all entries of the vector
        sumAll = 0
        for i in range(0,len(y)):
            sumAll = sumAll + y[i]
        return (z + sumAll + n - len(y)) * numPS(y, z, n)

In [16]:
def loc2pref(loc):
    '''
    Given one possible location sequence, output all possible preference vectors related. (Not necessarilly valid)
    >>> loc2pref([1,2,3])
    [[1, 1, 1], [1, 1, 2], [1, 1, 3], [1, 2, 1], [1, 2, 2], [1, 2, 3]]
    '''
    # preference sequence
    l = [list(range(1,a+1)) for a in loc]
    return [list(pref) for pref in product(*l)]

def len2pref(len_v,z):
    '''
    Given a length vector, output all possible preference vectors. (Not necessarily valid)
    '''
    locs = []
    prefs = []
    length = np.array(len_v)
    for p in permutations(range(0,len(length))):
        p = list(p)
        ip = np.argsort(p)
        loc = np.cumsum(length[p]) - length[p] + z
        loc_sorted = loc[list(ip)]
        locs.append(list(loc_sorted))
        prefs.append(loc2pref(loc_sorted))
    return locs, prefs

In [17]:
# a single simulation
def sim(lenV, z, locV, preV):
    '''
    input: length vector, z, location vector, preference vector
    output: a boolean value, true if the preference vector is a parking sequence
    '''
    num_car = len(lenV)
    num_lots = sum(lenV)+z-1
    lots = [0] * num_lots
    lots[0:z-1] = [1]*(z-1)
    for i in range(num_car):
        # initialize
        pref = preV[i]
        expected_loc = locV[i]
        le = lenV[i]
        actual_loc = 0
        while pref+le-1 <= num_lots:
#             print('pref:{}'.format(pref))
            # probe
            isVacant = (sum(lots[pref-1:pref+le-1])==0)
            if isVacant:
                # park
#                 print('park')
                lots[pref-1:pref+le-1] = [1]*len(lots[pref-1:pref+le-1])
                actual_loc = pref
                break
            elif (lots[pref-1]==0):
                # collision
#                 print('collision')
                break
            else:
                # keep going
#                 print('going')
                pref += 1
#         print('expect_loc:{}'.format(expected_loc))
#         print('actual_loc:{}'.format(actual_loc))
        if actual_loc != expected_loc:
            return False
    return True

# run simulation for all sequences
def runSim(lenV, z, locV, preV):
    '''
    start a new list to keep the valid preference vectors, i.e. parking sequences
    returns a PS list containing all valid parking sequences, and a dictionary matching each location to various valid parking sequences
    '''
    ps_dict = {}
    ps_list = []
    for i in range(len(locV)):
        for j in range(len(preV[i])):
            if sim(lenV, z, locV[i], preV[i][j]) == True:
                ps_dict.setdefault(str(locV[i]),[]).append(preV[i][j])
                ps_list.append(preV[i][j])
    return ps_dict,ps_list

In [18]:
# determine whether the given p sequence is a prime parking sequences
def prime(preV, y, z):
    prime = []
    y_inc = sorted(y)
    n = len(y)
    # check each preference vector in prefVec using corollary 2.2
    for i in range(len(preV)):
        c = sorted(preV[i])
        isPrime = True
        for j in range(n):
            # the first two smallest c should be no bigger than z
            if j == 0 and c[j] > z:
                isPrime = False
                break
            elif j == 1 and c[j] > z:
                isPrime = False
                break
            elif j >= 2:
                # starting from j = 2, c[j] is c_(3)
                bound = z + sum(y_inc[n-j+1:n])
                if c[j] > bound:
                    isPrime = False
                    break
        if isPrime:
            prime.append(preV[i])
    return prime


In [19]:
# invariance parking sequences
import itertools
def invariance(preV, y, z):
    invariance = []
    for vector in preV:
        # generate the permutations for the vector
        l = list(itertools.permutations(vector))
        # generates a list of tuples
        # if all the rearranged vectors are in preV, add the vector to invariance list
        inv = True
        for tpl in l:
            if list(tpl) not in preV:
                inv = False
        if inv == True:
            invariance.append(vector)
    return invariance

In [20]:
# generate brief version of invariance parking sequences
# this will enable us to find the pattern
def brief(inv):
    brief = inv.copy()
    # sort all vectors in increasing order
    for vector in brief:
        vector.sort()
    # remove duplicates
    mySet = set(tuple(x) for x in brief)
    return mySet

In [21]:
def easyRun(y,z,show_dict = False):
    '''Give all valid preference sequences given a length vector and trailer z.
    >>> easyRun(length_vector, trailer_z[, show_dict=False])
    setting show_dict to True will show the correspondence between the location and the preference vectors
    '''
    possibleLoc, possiblePref = len2pref(y,z)
    prefVec_dict, prefVec = runSim(y, z, possibleLoc, possiblePref)
    theo_num = numPS(y,z,len(y))
    primes = prime(prefVec, y, z)
    inv = invariance(prefVec, y, z)
    print(
        'Given length vector {} and z = {}'.format(y,z),
#         'All {} possible location sequences:'.format(len(possibleLoc)),
#         possibleLoc,
    #     'All possible preference sequences:',
    #     possiblePref,
        'Theory suggests that there will be {} valid preference vectors.'.format(theo_num),
        'The simulation gives {} valid preference vectors:'.format(len(prefVec)),
        prefVec,
        '{} of the parking sequences are prime.'.format(len(primes)),
        primes,
        '{} of the parking sequences are invariant.'.format(len(inv)),
        inv,
        sep = '\n'
    )
    brf = brief(inv)
    print(
        'They are permutations of each of these sets: ',
        brf,
        sep = '\n'
    )
    if show_dict:
        # show the correspondence between the location and the preference vectors
        for key, value in prefVec_dict.items():
            print('Location: {} \nPreferences: {}'.format(key,value), end='\n\n')
    return None

## Examples Below

In [22]:
length_vector = [3,3,3,2]
trailer_z = 1
easyRun(length_vector, trailer_z)

Given length vector [3, 3, 3, 2] and z = 1
Theory suggests that there will be 693 valid preference vectors.
The simulation gives 693 valid preference vectors:
[[1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 1, 3], [1, 1, 1, 4], [1, 1, 1, 5], [1, 1, 1, 6], [1, 1, 1, 7], [1, 1, 1, 8], [1, 1, 1, 9], [1, 1, 1, 10], [1, 1, 2, 1], [1, 1, 2, 2], [1, 1, 2, 3], [1, 1, 2, 4], [1, 1, 2, 5], [1, 1, 2, 6], [1, 1, 2, 7], [1, 1, 2, 8], [1, 1, 2, 9], [1, 1, 2, 10], [1, 1, 3, 1], [1, 1, 3, 2], [1, 1, 3, 3], [1, 1, 3, 4], [1, 1, 3, 5], [1, 1, 3, 6], [1, 1, 3, 7], [1, 1, 3, 8], [1, 1, 3, 9], [1, 1, 3, 10], [1, 1, 4, 1], [1, 1, 4, 2], [1, 1, 4, 3], [1, 1, 4, 4], [1, 1, 4, 5], [1, 1, 4, 6], [1, 1, 4, 7], [1, 1, 4, 8], [1, 1, 4, 9], [1, 1, 4, 10], [1, 1, 5, 1], [1, 1, 5, 2], [1, 1, 5, 3], [1, 1, 5, 4], [1, 1, 5, 5], [1, 1, 5, 6], [1, 1, 5, 7], [1, 1, 5, 8], [1, 1, 5, 9], [1, 1, 5, 10], [1, 1, 6, 1], [1, 1, 6, 2], [1, 1, 6, 3], [1, 1, 6, 4], [1, 1, 6, 5], [1, 1, 6, 6], [1, 1, 6, 7], [1, 1, 6, 8], [1, 1, 6, 9], [1, 1, 6,

In [29]:
length_vector = [5,5,5,3,3]
trailer_z = 1
easyRun(length_vector, trailer_z)

Given length vector [5, 5, 5, 3, 3] and z = 1
Theory suggests that there will be 50400 valid preference vectors.
The simulation gives 50400 valid preference vectors:
7244 of the parking sequences are prime.
66 of the parking sequences are invariant.
[[1, 1, 1, 1, 1], [1, 1, 1, 1, 6], [1, 1, 1, 1, 11], [1, 1, 1, 1, 14], [1, 1, 1, 1, 6], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 6, 14], [1, 1, 1, 1, 11], [1, 1, 1, 6, 11], [1, 1, 1, 1, 14], [1, 1, 1, 6, 14], [1, 1, 1, 1, 6], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 6, 14], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 6, 14], [1, 1, 1, 1, 11], [1, 1, 1, 6, 11], [1, 1, 1, 6, 11], [1, 1, 1, 1, 6], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 6, 14], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 6, 14], [1, 1, 1, 6, 6], [1, 1, 1, 6, 11], [1, 1, 1, 1, 14], [1, 1, 1, 6, 14], [1, 1, 1, 6, 14], [1, 1, 1, 6, 14], [1, 1, 1, 1, 11], [1, 1, 1, 6, 11], [1, 1, 1, 6, 11], [1, 1, 1, 6, 11], [1, 1, 1, 1, 14], [1, 1, 1, 6, 14], [1, 1, 1, 6, 14], [1, 